In [1]:
# -*- coding: utf-8 -*-
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [2]:
driver = webdriver.Chrome('C:/chromedriver.exe')
driver.get("http://www.airportal.go.kr/life/airinfo/RbHanFrm.jsp")

In [3]:
def setDate(datevalue):
    date = driver.find_element_by_xpath('//*[@id="current_date"]')
    date.click()
    date.clear()
    date.click()
    date.send_keys(str(datevalue))

In [4]:
def setAirport(airportvalue):
    airport = driver.find_element_by_xpath('/html/body/form/table/tbody/tr/td[2]/table/tbody/tr[3]/td/table/tbody/tr[2]/td/table/tbody/tr[1]/td[4]/span/select/option['+str(airportvalue)+']').click()

In [5]:
def setDivision(d):
    if d==0:
        path = '/html/body/form/table/tbody/tr/td[2]/table/tbody/tr[3]/td/table/tbody/tr[2]/td/table/tbody/tr[1]/td[2]/input[1]'
    else:
        path = '/html/body/form/table/tbody/tr/td[2]/table/tbody/tr[3]/td/table/tbody/tr[2]/td/table/tbody/tr[1]/td[2]/input[2]'
        
    driver.find_element_by_xpath(path).click()

In [6]:
def Search():
    driver.find_element_by_xpath("/html/body/form/table/tbody/tr/td[2]/table/tbody/tr[3]/td/table/tbody/tr[2]/td/table/tbody/tr[2]/td[4]/a/img").click()

In [7]:
def Data(d,airportvalue,date):
    #d=0이면 도착
    setDivision(d)
    #airportvalue==2 : 김포 or airportvalue==8 : 제주 
    setAirport(airportvalue)
    setDate(date)
    Search()

In [8]:
Data(1,8,20190901)

In [41]:
l = driver.find_element_by_xpath('/html/body/form/table/tbody/tr/td[2]/table/tbody/tr[5]/td/iframe')

In [42]:
driver.switch_to_frame(l)

c:\users\kss09\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: use driver.switch_to.frame instead
  """Entry point for launching an IPython kernel.


In [45]:
li = [i.split() for i in driver.find_element_by_tag_name("table").text.split('\n')]

In [48]:
print(li)

[['아시아나항공', 'OZ8900', 'GMP(김포)', '06:30', '06:30', '06:43', '여객', '출발'], ['아시아나항공', 'OZ8902', 'GMP(김포)', '06:40', '06:40', '06:52', '여객', '출발'], ['대한항공', 'KE1200', 'GMP(김포)', '06:40', '06:40', '06:59', '여객', '출발'], ['대한항공', 'KE1202', 'GMP(김포)', '07:00', '07:00', '07:14', '여객', '출발'], ['춘추항공', '9C6150', 'NKG(남경)', '07:00', '07:00', '07:08', '여객', '출발'], ['진에어', 'LJ302', 'GMP(김포)', '07:05', '07:05', '07:20', '여객', '출발'], ['춘추항공', '9C8624', 'HGH(항저우)', '07:25', '07:25', '07:28', '여객', '출발'], ['홍콩익스프레스', 'UO697', 'HKG(홍콩)', '07:30', '07:30', '07:40', '여객', '출발'], ['춘추항공', '9C8708', 'SHE(심양)', '07:30', '07:30', '07:41', '여객', '출발'], ['대한항공', 'KE1802', 'TAE(대구)', '07:35', '07:35', '07:47', '여객', '출발'], ['제주항공', '7C140', 'GMP(김포)', '07:40', '07:40', '08:02', '여객', '출발'], ['아시아나항공', 'OZ8904', 'GMP(김포)', '07:55', '07:55', '08:16', '여객', '출발'], ['제주항공', '7C852', 'CJJ(청주)', '07:55', '07:55', '08:19', '여객', '출발'], ['티웨이항공', 'TW702', 'GMP(김포)', '07:55', '07:55', '08:09', '여객', '출발'], ['진에어', 'LJ304